## Load mosquito data

In [9]:
import pandas as pd

# Set the display option to show all columns
# pd.set_option('display.max_columns', None)

# Read the txt file into a pandas DataFrame
data = pd.read_csv('../data/raw_data/West_Nile_Virus__WNV__Mosquito_Test_Results.csv')

# Display the first few rows of the DataFrame
data.head()





,SEASON YEAR,WEEK,TEST ID,BLOCK,TRAP,TRAP_TYPE,TEST DATE,NUMBER OF MOSQUITOES,RESULT,SPECIES,LATITUDE,LONGITUDE,LOCATION
0,2021,22,51815,100XX W OHARE,T909,GRAVID,6/3/21 00:06,19,negative,CULEX PIPIENS/RESTUANS,3.0,4.0,NaN
1,2021,22,51816,100XX W OHARE,T909,GRAVID,6/3/21 00:06,5,negative,CULEX RESTUANS,NaN,NaN,NaN
2,2021,23,51918,100XX W OHARE,T909,GRAVID,6/10/21 00:06,50,negative,CULEX PIPIENS/RESTUANS,NaN,NaN,NaN
3,2021,33,52988,100XX W OHARE,T909,GRAVID,8/19/21 00:08,50,negative,CULEX PIPIENS/RESTUANS,NaN,NaN,NaN
4,2022,23,53486,100XX W OHARE,T904,GRAVID,6/10/22 00:06,23,negative,CULEX PIPIENS/RESTUANS,NaN,NaN,NaN


## identify missing data

In [11]:
missing_values_count = data.isnull().sum()
missing_values_count

SEASON YEAR                0
WEEK                       0
TEST ID                    0
BLOCK                      0
TRAP                       0
TRAP_TYPE                  0
TEST DATE                  0
NUMBER OF MOSQUITOES       0
RESULT                     0
SPECIES                    0
LATITUDE                5318
LONGITUDE               5318
LOCATION                5319
dtype: int64

## find the missing lat/long and group them by block

In [19]:
missing_lat_long_data = data[data['LATITUDE'].isnull() | data['LONGITUDE'].isnull()]

# Group by 'BLOCK', count the missing values and list the 'TEST DATE' for each
block_missing_summary = missing_lat_long_data.groupby('BLOCK').agg(
    Missing_Count=pd.NamedAgg(column='LATITUDE', aggfunc='size'), # Count of missing values
    Test_Dates=pd.NamedAgg(column='TEST DATE', aggfunc=lambda x: x.unique().tolist()) # Unique test dates
).reset_index()

# Display the summary
block_missing_summary

,BLOCK,Missing_Count,Test_Dates
0,100XX W OHARE,500,"[6/3/21 00:06, 6/10/21 00:06, 8/19/21 00:08, 6..."
1,100XX W OHARE AIRPORT,3426,"[6/16/17 00:06, 6/3/21 00:06, 6/8/18 00:06, 6/..."
2,115XX S AVENUE L,80,"[8/28/07 00:08, 8/1/07 02:08, 10/4/07 00:10, 8..."
3,20XX N DOMINICK ST,142,"[7/19/18 00:07, 7/3/19 00:07, 6/12/20 00:06, 8..."
4,30XX S HOYNE,24,"[9/18/07 00:09, 7/27/07 11:07, 8/21/07 00:08, ..."
5,43XX N ASHLAND,15,"[8/21/07 00:08, 9/18/07 00:09, 6/26/07 04:06, ..."
6,4XX W 127TH,512,"[6/16/17 00:06, 8/3/17 00:08, 6/12/20 00:06, 6..."
7,65XX N OAK PARK AVE,31,"[8/15/07 00:08, 10/4/07 00:10, 6/5/07 00:06, 6..."
8,79XX S CHICAGO,298,"[8/3/17 00:08, 6/10/22 00:06, 6/13/19 00:06, 6..."
9,81XX S ASHLAND,269,"[8/3/17 00:08, 6/19/17 00:06, 6/8/18 00:06, 7/..."


### This code was used to check to see if there were any lat/long listed for at least one of the block records. To check if this code works, I added fake data in two fields. 

In [22]:
# Extract the list of blocks with missing lat/long data
blocks_with_missing_values = block_missing_summary['BLOCK'].unique()

# Filter the original dataset for entries that are in the list of blocks with missing values
# but have valid latitude and longitude data
blocks_with_valid_lat_long = data[
    data['BLOCK'].isin(blocks_with_missing_values) &
    data['LATITUDE'].notnull() &
    data['LONGITUDE'].notnull()
]

# Group by 'BLOCK' and list the unique latitude and longitude values for these entries
block_valid_lat_long_summary = blocks_with_valid_lat_long.groupby('BLOCK').agg({
    'LATITUDE': lambda x: x.unique().tolist(),
    'LONGITUDE': lambda x: x.unique().tolist()
}).reset_index()

# The resulting DataFrame 'block_valid_lat_long_summary' will contain each block along with
# the associated valid latitude and longitude values that exist in the dataset.
block_valid_lat_long_summary


,BLOCK,LATITUDE,LONGITUDE
0,100XX W OHARE,[3.0],[4.0]


### Because I am able to get the lat and long for these blocks, I will keep them and dropp all the others.

In [25]:
# Define the blocks to keep even if they have missing lat/long
blocks_to_keep = ['100XX W OHARE', '100XX W OHARE AIRPORT', '4XX W 127TH']

# Filter the data to exclude rows with missing lat/long unless the block contains one of the specified blocks to keep
data_filtered = data[
    (~data['LATITUDE'].isnull() & ~data['LONGITUDE'].isnull()) |  # Keep rows with valid lat/long
    (data['BLOCK'].str.contains('|'.join(blocks_to_keep)))  # Or rows that contain the specified blocks
]

# The resulting DataFrame 'data_filtered' will have the rows with missing values dropped,
# except for the specified blocks.

Adding Lat and long for the 3 blocks and saving new dataset to the processed data folder

In [28]:
# Assuming your dataframe is named data_filtered
# Update the lat/long values for the specified locations
data_filtered.loc[data_filtered['BLOCK'] == '100XX W OHARE', ['LATITUDE', 'LONGITUDE']] = 41.978611, -87.904724
data_filtered.loc[data_filtered['BLOCK'] == '100XX W OHARE AIRPORT', ['LATITUDE', 'LONGITUDE']] = 41.978611, -87.904724
data_filtered.loc[data_filtered['BLOCK'] == '4XX W 127TH', ['LATITUDE', 'LONGITUDE']] = 41.66318849, -87.63267836

# Save the updated dataframe to a new CSV file in the same directory
data_filtered.to_csv('../data/processed_data/wnv_cleaned.csv', index=False)

## Load weather data

In [31]:
import pandas as pd

# Set the display option to show all columns
#pd.set_option('display.max_columns', None)
# Read the txt file into a pandas DataFrame
wx = pd.read_csv('../data/raw_data/ORD_weather.csv')

# Display the first few rows of the DataFrame
wx.head()

,STATION,NAME,DATE,PRCP,TMAX
0,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",1/30/11,0.00,30
1,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",1/31/11,0.09,26
2,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2/1/11,0.74,23
3,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2/2/11,0.74,23
4,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2/3/11,0.00,16


### identify missing data

There are no missing values so we can proceed with the next step. date ranges


In [35]:
#date range for the mosquito dataset
year_min_mos = data['SEASON YEAR'].min()
year_max_mos = data['SEASON YEAR'].max()

#date range for the weather datasetab
year_min_wx = wx['DATE'].min()
year_max_wx = wx['DATE'].max()

print(f"The range of the mosquito dataset is {year_min_mos} to {year_max_mos}")
print(f"The range of the weather dataset is {year_min_wx} to {year_max_wx}")



The range of the mosquito dataset is 2007 to 2023
The range of the weather dataset is 1/1/12 to 9/9/23


In [37]:
# Corrected file path
file_path = '../data/processed_data/wnv_cleaned.csv'

# Read the CSV file
mos_clean = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
mos_clean.head()


,SEASON YEAR,WEEK,TEST ID,BLOCK,TRAP,TRAP_TYPE,TEST DATE,NUMBER OF MOSQUITOES,RESULT,SPECIES,LATITUDE,LONGITUDE,LOCATION
0,2021,22,51815,100XX W OHARE,T909,GRAVID,6/3/21 00:06,19,negative,CULEX PIPIENS/RESTUANS,41.978611,-87.904724,NaN
1,2021,22,51816,100XX W OHARE,T909,GRAVID,6/3/21 00:06,5,negative,CULEX RESTUANS,41.978611,-87.904724,NaN
2,2021,23,51918,100XX W OHARE,T909,GRAVID,6/10/21 00:06,50,negative,CULEX PIPIENS/RESTUANS,41.978611,-87.904724,NaN
3,2021,33,52988,100XX W OHARE,T909,GRAVID,8/19/21 00:08,50,negative,CULEX PIPIENS/RESTUANS,41.978611,-87.904724,NaN
4,2022,23,53486,100XX W OHARE,T904,GRAVID,6/10/22 00:06,23,negative,CULEX PIPIENS/RESTUANS,41.978611,-87.904724,NaN


In [50]:
# Filter the DataFrame to keep only the records where 'SEASON YEAR' is >= 2011
mos_trim = mos_clean[mos_clean['SEASON YEAR'] >= 2011]

# Find the minimum value of 'SEASON YEAR' in the filtered DataFrame
min_season_year = mos_trim['SEASON YEAR'].min()

min_season_year

mos_trim.to_csv('../data/processed_data/wnv_11_23.csv', index=False)

In [41]:
import pdfplumber
import pandas as pd

# Path to the PDF file
pdf_path = '../data/raw_data/FL2011week52arbovirusreport-dec31.pdf'

tables = []  # To store all the tables extracted

# Open the PDF file
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[0]  # Assuming the table is on the first page
    table = page.extract_table()  # Extract the table
    if table:  # If a table is found
        tables.append(pd.DataFrame(table[1:], columns=table[0]))

# Display the first table as a DataFrame
if tables:
    display(tables[0])



,Florida Arbovirus Surveillance
0,"Week 52: December 25 – December 31, 2011"


In [43]:
import pandas as pd

# Assuming df is your DataFrame after reading the CSV
df = pd.read_csv('../data/processed_data/wnv_trim.csv')

# Group the data by 'season year', 'block', 'week', and 'result' to count occurrences
grouped = df.groupby(['SEASON YEAR', 'BLOCK', 'WEEK', 'RESULT']).size().reset_index(name='count')

# Pivot the table to have 'result' as columns ('positive', 'negative') and counts as values
pivot_table = grouped.pivot_table(index=['SEASON YEAR', 'BLOCK', 'WEEK'], columns='RESULT', values='count', fill_value=0).reset_index()

# Rename columns if necessary (e.g., to 'Number of Negative Pools', 'Number of Positive Pools')
pivot_table.columns = ['SEASON YEAR', 'BLOCK', 'WEEK', 'Number of Negative Pools', 'Number of Positive Pools']

# Display the new table
print(pivot_table.head())

# Let's save this DataFrame to a new CSV file

pivot_table.to_csv('../data/processed_data/wnv_result_summary.csv', index=False)

# Providing the path to the newly saved CSV file
'../data/processed_data/wnv_result_summary.csv'




   SEASON YEAR          BLOCK  WEEK  Number of Negative Pools  \
0         2018  100XX W OHARE    23                         1   
1         2018  100XX W OHARE    24                         2   
2         2018  100XX W OHARE    25                         2   
3         2018  100XX W OHARE    26                         5   
4         2018  100XX W OHARE    27                         1   

   Number of Positive Pools  
0                         0  
1                         0  
2                         0  
3                         1  
4                         0  


'../data/processed_data/wnv_result_summary.csv'

In [45]:
import pandas as pd

# Step 1: Load the dataset
df_wx = pd.read_csv('../data/raw_data/ORD_weather.csv')

# Ensure the DATE column is in datetime format
df_wx['DATE'] = pd.to_datetime(df_wx['DATE'])

# Step 2: Create a 'Year-Week' column in 'YYYY-WW' format
df_wx['Year'] = df_wx['DATE'].dt.year
df_wx['Week'] = df_wx['DATE'].dt.isocalendar().week
df_wx['Year-Week'] = df_wx['Year'].astype(str) + '-' + df_wx['Week'].apply(lambda x: f'{x:02d}')

# Step 3: Group by the new 'Year-Week' column and calculate the desired metrics
weekly_data = df_wx.groupby('Year-Week').agg(
    Avg_TMAX=('TMAX', 'mean'),  # Calculate average of TMAX
    Total_PRCP=('PRCP', 'sum')  # Calculate total of PRCP
).reset_index()

# Display the resulting DataFrame
print(weekly_data)

# Save the DataFrame to the specified location
weekly_data.to_csv('../data/processed_data/wx_weeknum.csv', index=False)



/var/folders/98/yljxdbwn1zl6kzb6gbpk54xh0000gn/T/ipykernel_42531/1301416414.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_wx['DATE'] = pd.to_datetime(df_wx['DATE'])


    Year-Week   Avg_TMAX  Total_PRCP
0     2011-04  30.000000        0.00
1     2011-05  24.714286        1.77
2     2011-06  27.428571        0.13
3     2011-07  43.714286        0.95
4     2011-08  33.428571        0.62
..        ...        ...         ...
621   2023-48  38.714286        0.94
622   2023-49  45.285714        0.36
623   2023-50  44.142857        0.41
624   2023-51  41.400000        0.51
625   2023-52  45.000000        0.16

[626 rows x 3 columns]


In [47]:
import os

# Check the current working directory
print("Current Working Directory:", os.getcwd())

# If needed, change to the directory where your notebook should be running from
# os.chdir('your/base/directory')


Current Working Directory: /Users/andrewruiz/HPAM9000_ruiz/Python
